In [2]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()


# things we need for Tensorflow


import numpy as np
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
# import our chat-bot intents file
import json
with open('intentBotData.json') as json_data:
    intents = json.load(json_data)

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)
print(documents)

34 documents
10 classes ['Does the screen state BAD I/O?', 'Gate Issues', 'Have you been here more than 3 hours?', 'How To Purchase A Pass', 'Lost/Missing Ticket', 'Machine Issues', 'Other Issues', 'Pass Is Unreadable', 'Payment Issues', 'Ticket Is Unreadable']
89 unique stemmed words ["'", "'paid", "'ve", ',', '3', 'a', 'acceiv', 'access', 'and', 'bad', 'banknote/bill', 'been', 'but', 'buy', 'ca', 'can', 'card', 'chang', 'charg', 'credit', 'dam', 'delivery', 'diff', 'dispens', 'do', 'doe', 'entry', 'er', 'excess', 'find', 'for', 'gat', 'giv', 'has', 'hav', 'hear', 'her', 'hour', 'how', 'i', 'i/o', 'if', 'in', 'insert', 'is', 'it', 'limit', 'lost', 'machin', 'machine/vandalism', 'mess', 'miss', 'month', 'my', "n't", 'no', 'not', 'on', 'op', 'operator/operat', 'out', 'ov', 'paid', 'park', 'pass', 'plac', 'purchas', 'read', 'receipt', 'return', 'say', 'scan', 'screen', 'should', 'stuck', 'than', 'the', 'ticket', 'tim', 'to', 'unread', 'us', 'was', 'week', 'what', 'wil', 'without', 'wo', 

In [5]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print(np.array(train_x).shape)
print(np.array(train_y).shape)

(34, 89)
(34, 10)


C:\Users\gurda\AppData\Local\Temp\ipykernel_29300\3838598322.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [6]:
# reset underlying graph data
# tf.reset_default_graph()
# Build neural network
#https://towardsdatascience.com/building-your-first-neural-network-in-tensorflow-2-tensorflow-for-hackers-part-i-e1e2f1dfe7a0

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 4999  | total loss: 0.00309 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.00309 - acc: 1.0000 -- iter: 32/34
Training Step: 5000  | total loss: 0.00300 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.00300 - acc: 1.0000 -- iter: 34/34
--
INFO:tensorflow:c:\Aspire_Data\ClassAssignment\Semester-3\Capstone\Test_Project_Implement\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )


In [8]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('IntentBotData.json') as json_data:
    intents = json.load(json_data)

In [9]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from c:\Aspire_Data\ClassAssignment\Semester-3\Capstone\Test_Project_Implement\model.tflearn


In [10]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [11]:
ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [12]:
classify('How to buy a pass')

[('How To Purchase A Pass', 0.9991309)]

In [13]:
response('How to buy a Pass?')


To purchase a pass without a ticket, press the 'Pass' button on the screen. You can change the pass type to daily, weekly, or monthly by pressing the 'Next' button.


In [14]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.50
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))
                    
                    if i['tag'] == results[0][0]: 
                        if 'context_set' in i:
                            if show_details: print ('context:', i['context_set'])
                            context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [21]:
response('ticket lost')

If you've lost your ticket, you can purchase a replacement from the pay station. Please note that you will be charged for the maximum time.


In [16]:
response('Whats your name')

In [20]:
response('I lost my tickert')


If you've lost your ticket, you can purchase a replacement from the pay station. Please note that you will be charged for the maximum time.


In [ ]:
# Evaluating Model

# Accuracy
